# Running a 5-Point Calibration Scan With OSO

This notebook adapts the Five Point Calibration Scan notebook to work with PSI. Running this notebook requires that a PSI namespace be spun up with `DISH_LMC_ENABLED` set to true.

This notebook can be Run with BITE or SPFRx, with BITE requiring the optional BITE notebook to be run. 

###  For using BITE

 BITE Functionality has been spun out to a separate notebook, which should be referred to when needed. These sections are labeled as BITE STEP.

## 1 Setup 

### 1.1 Environment Setup

Start by importing all the libraries needed to run this notebook, including local ones.

In [1]:
import sys
import re

sys.path.append("../../src")

import json
import os
from time import sleep

from ska_oso_pdm import SBDefinition
from ska_oso_scripting import oda_helper
from ska_oso_scripting.functions import pdm_transforms
from ska_oso_scripting.functions.sb import create_sbi, load_sbd
from ska_oso_scripting.objects import SubArray, Telescope
from tango import DevFailed, DeviceProxy

### 1.2 Set Variables

First, grab the namespace launched from the pipeline:

In [2]:
!kubectl get ns | grep ska-mid-psi 

ci-ska-mid-psi-1406480615-alexschell                     Active   3m25s
ci-ska-mid-psi-1406480615-alexschell-sdp                 Active   3m25s


And load it into the variables for the notebook:

In [3]:
namespace = "ci-ska-mid-psi-1406480615-alexschell"  # set to desired NS
simulation_mode = 0  # set to 1 to run in sim mode
target_boards_list = [1]  # assign boards
scheduling_block_definition = "mid_5_point_sb.json"  # SB def file in data/scheduling_block to use
server = "ska-sdp-kafka." + namespace + ".svc.cluster.local:9092"
subarray_id = 1

With the namespace set, use these links to monitor the system as the notebook is worked through

In [4]:
print("Monitor signal received by the vis pod:")
print(f"https://142.73.34.170/{namespace}/signal/display/")
print("Dashboard for monitoring device status:")
print(f"https://142.73.34.170/{namespace}/taranta/dashboard?id=666cb28b5e5d4f0012197e5f&mode=run")
print("Dashboard for monitoring TMC status:")
print(f"https://142.73.34.170/{namespace}/taranta/dashboard?id=669ea8e32bc4790019e64bba&mode=run")
print("PgAdmin interface to monitor the databases and ensure SB/EB are properly set in them")
print(f"https://142.73.34.170/{namespace}/pgadmin4/browser/")

Monitor signal received by the vis pod:
https://142.73.34.170/ci-ska-mid-psi-1406480615-alexschell/signal/display/
Dashboard for monitoring device status:
https://142.73.34.170/ci-ska-mid-psi-1406480615-alexschell/taranta/dashboard?id=666cb28b5e5d4f0012197e5f&mode=run
Dashboard for monitoring TMC status:
https://142.73.34.170/ci-ska-mid-psi-1406480615-alexschell/taranta/dashboard?id=669ea8e32bc4790019e64bba&mode=run
PgAdmin interface to monitor the databases and ensure SB/EB are properly set in them
https://142.73.34.170/ci-ska-mid-psi-1406480615-alexschell/pgadmin4/browser/


Get the SKUID pod:

In [5]:
!kubectl get svc -n $namespace | grep skuid
!kubectl get svc -n $namespace | grep kafka

ska-ser-skuid-test-1406480615-svc               ClusterIP      10.111.78.142    <none>           9870/TCP                                          2m36s
ska-sdp-kafka                                   ClusterIP      10.110.242.133   <none>           9092/TCP                                          2m36s
ska-sdp-kafka-headless                          ClusterIP      None             <none>           9092/TCP,9094/TCP,9093/TCP                        2m36s


And set the id value based on the name of this pod:

In [6]:
skuid_id = "ska-ser-skuid-test-1406480615-svc"
kafka_id = "ska-sdp-kafka"

As a work around step for SKUID pod access, get the IP from the pod, get the IP address of the SKUID pod then set that as our access url:

In [7]:
skuid_name = !kubectl -n $namespace get pods | grep ska-ser-skuid-test
skuid_name = skuid_name[0].split()[0]
print(skuid_name)
extracted_ip = !kubectl -n $namespace describe pod $skuid_name | grep ips -A 1
ip = re.search("[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}", str(extracted_ip))
print(ip.group(0))

kafka_name = !kubectl -n $namespace get pods | grep ska-sdp-kafka
kafka_name = kafka_name[0].split()[0]
print(kafka_name)
extracted_kafka_ip = !kubectl -n $namespace describe pod $kafka_name | grep ips -A 1
kafka_ip = re.search("[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}\.[0-9]{0,3}", str(extracted_kafka_ip))
print(kafka_ip.group(0))

ska-ser-skuid-test-1406480615-74bbb4f6dd-7552g
10.10.212.246
ska-sdp-kafka-0
10.10.212.233


Copy the IP from this store it to be written later:

In [8]:
SKUID_POD = ip.group(0) + ":9870"
KAFKA_POD = kafka_ip.group(0) + ":9092"

Next, load all the other vars the notebook will use. These should not need to be changed for this run.

In [9]:
# Tango host environment variable
TANGO_HOST = "databaseds-tango-base." + namespace + ".svc.cluster.local:10000"
# Don't use for now...
# SKUID_POD = skuid_id + "." + namespace + ".svc.cluster.local:5004"

# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")
# Config file directories
COMMON_CONFIG = os.path.join(DATA_DIR, "psi/cbf")
CSP_CONFIG = os.path.join(DATA_DIR, "psi/csp")
TMC_CONFIG = os.path.join(DATA_DIR, "psi/tmc")
HW_CONFIG = os.path.join(DATA_DIR, "psi/cbf/hw_config")
SLIM_CONFIG = os.path.join(COMMON_CONFIG, "slim_config")
CBF_INPUT_DIR = os.path.join(COMMON_CONFIG, "cbf_input_data")
SB_DIR = os.path.join(DATA_DIR, "scheduling_block")

# For mapping the talon boards to receptor
RECEPTOR_MAP = ["SKA001", "SKA036", "SKA063", "SKA100"]

Next, set the environment arg for TANGO HOST, SKUID_POD and KAFKA, along with the oda API service endpoint:

In [10]:
os.environ["TANGO_HOST"] = TANGO_HOST
os.environ["SKUID_URL"] = SKUID_POD
os.environ["ODA_URL"] = f"http://142.73.34.170/{namespace}/oda/api/v5/"
os.environ["telescope"] = "mid"
print("SKUID service set to: ", os.environ["SKUID_URL"])
print("TANGO_HOST set to:", os.environ["TANGO_HOST"])
print("ODA endpoint set to:{}".format(os.environ["ODA_URL"]))
print("This can be checked via {}ui".format(os.environ["ODA_URL"]))

SKUID service set to:  10.10.212.246:9870
TANGO_HOST set to: databaseds-tango-base.ci-ska-mid-psi-1406480615-alexschell.svc.cluster.local:10000
ODA endpoint set to:http://142.73.34.170/ci-ska-mid-psi-1406480615-alexschell/oda/api/v5/
This can be checked via http://142.73.34.170/ci-ska-mid-psi-1406480615-alexschell/oda/api/v5/ui


With all the file paths defined the JSON files can be loaded in and checked.

In [11]:
print("Getting files...")

INIT_SYS_PARAM_FILE = os.path.join(COMMON_CONFIG, "sys_params/initial_system_param.json")
ASSIGN_RESOURCES_FILE = os.path.join(TMC_CONFIG, "assign_resources.json")
CONFIGURE_SCAN_FILE = os.path.join(TMC_CONFIG, "configure_scan.json")
SCAN_FILE = os.path.join(TMC_CONFIG, "scan.json")

CBF_INPUT_FILE = f"{CBF_INPUT_DIR}/cbf_input_data.json"
BITE_CONFIG_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/bite_configs.json"
FILTERS_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/filters.json"

DISH_CONFIG_FILE = f"{COMMON_CONFIG}/sys_params/load_dish_config.json"

SB_FILE = os.path.join(SB_DIR, scheduling_block_definition)

SCAN_COMBOS = [[0.0, 5.0], [0.0, -5.0], [5.0, 0.0], [-5.0, 0.0]]

files = [
    INIT_SYS_PARAM_FILE,
    ASSIGN_RESOURCES_FILE,
    CONFIGURE_SCAN_FILE,
    SCAN_FILE,
    CBF_INPUT_FILE,
    BITE_CONFIG_FILE,
    FILTERS_FILE,
    DISH_CONFIG_FILE,
    SB_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

Getting files...
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/psi/cbf/sys_params/initial_system_param.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/psi/tmc/assign_resources.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/psi/tmc/configure_scan.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/psi/tmc/scan.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/psi/cbf/cbf_input_data/cbf_input_data.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/psi/cbf/cbf_input_data/bite_config_parameters/bite_configs.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/psi/cbf/cbf_input_data/bite_config_parameters/filters.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/psi/cbf/sys_params/load_dish_config.json exists: ✔️
/home/al097049_p/SKAcode/ska-mid-jupyter-notebooks/data/scheduling_block/mid_5_point_sb.json exists: ✔️


Next, set up the hw config to match the boards in use:

In [12]:
if any(i > 4 for i in target_boards_list):
    print("Using swap for higher number talons")
    config = "hw_config_swap_psi.yaml"
    print("Modifying target to use lower nums to match swap file")
    target_boards_list = list(map(lambda x: x - 4, target_boards_list))

else:
    print("Using standard HW config")
    config = "hw_config_psi.yaml"

HW_CONFIG_FILE = os.path.join(HW_CONFIG, config)
if os.path.isfile(HW_CONFIG_FILE):
    print("HW config: ✔️")
else:
    print("hw config: ❌")

receptor_ids = list(map(lambda x: RECEPTOR_MAP[x - 1], target_boards_list))

Using standard HW config
HW config: ✔️


If the HW config file exists, it is loaded into the pod:

In [13]:
!kubectl cp $HW_CONFIG_FILE $namespace/ds-cbfcontroller-controller-0:/app/mnt/hw_config/hw_config.yaml 

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Set Variables in the BITE Notebook</h3>
    If running the BITE notebook, at this point ensure the variables in step 1.2 are set, and match the ones set here.
</div>
</body>

### 1.3 Create Device Proxies

With the pod spun up, create device proxies to the devices used and check the connection to them.

In [14]:
# CSP Devices
csp_controller = DeviceProxy("mid-csp/control/0")
print(f"CSP Controller: {csp_controller.Status()}")

# TMC Devices
tmc_central_node = DeviceProxy("ska_mid/tm_central/central_node")
print(f"Central Node: {tmc_central_node.Status()}")
tmc_subarray = DeviceProxy("ska_mid/tm_subarray_node/1")
print(f"TMC subarray Node: {tmc_subarray.Status()}")

# For checking leaf node
leaf_node_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")

# Deployer for setup and BITE for data mocking
deployer = DeviceProxy("mid_csp_cbf/ec/deployer")

# CBF Device
cbf_subarray = DeviceProxy("mid_csp_cbf/sub_elt/subarray_01")
print(f"CBF subarray: {cbf_subarray.Status()}")

CSP Controller: The device is in DISABLE state.
Central Node: The device is in ON state.
TMC subarray Node: The device is in ON state.
CBF subarray: The device is in DISABLE state.


### 1.4 Downloading Requirements via the Deployer

First, set the board to deploy to and turn on the deployer device.

In [15]:
deployer.targetTalons = target_boards_list
print("Deployer will target board:", deployer.targetTalons)
deployer.generate_config_jsons()

Deployer will target board: [1]


Once started and configured, the required devices can then be downloaded.

In [16]:
deployer.set_timeout_millis(400000)
try:
    deployer.download_artifacts()
except DevFailed as e:
    print(e)
    print(
        "Timed out, this is likely due to the download taking some time. Check the logs with the code space below after some time to see if it passes."
    )
deployer.set_timeout_millis(3000)

Now configure the device database with the downloaded devices:

In [17]:
deployer.configure_db()

### 1.5 Set up Execution Block

The first step is to generate an empty Execution Block, using the helper scripts to work with the API endpoint:

In [18]:
exec_block_id = oda_helper.create_eb(telescope="ska_mid")
print(exec_block_id)

eb-t0001-20240814-00002


## 2 Preparing Telescope

Start by setting the admin modes to allow for control of the telescope:

In [19]:
csp_controller.write_attribute("adminMode", 0)
tmc_central_node.write_attribute("adminMode", 0)
# Set CBF Simulation mode to false and CBF timeout to 99s
sleep(2)
csp_controller.cbfSimulationMode = 0
csp_controller.commandTimeout = 99

Ensure that these are set correctly, admin mode should be 0 (online) and cbfSimulationMode should be FALSE.

In [20]:
print(csp_controller.adminMode)
print(tmc_central_node.adminMode)
print(csp_controller.cbfSimulationMode)

adminMode.ONLINE
adminMode.ONLINE
False


Start by loading in the VCC config (this step may need to be re-run):

In [22]:
with open(DISH_CONFIG_FILE, encoding="utf-8") as f:
    dish_config_json = json.load(f)

dish_config_json["tm_data_sources"][
    0
] = "car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"
dish_config_json["tm_data_filepath"] = "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"

print(f"dish_config_json file contents: \n{dish_config_json}")
tmc_central_node.LoadDishCfg(json.dumps(dish_config_json))

sleep(2)
print(f"TMC CSP Master's Dish Vcc Config attribute value: \n{leaf_node_master.dishVccConfig}")
print(
    f"\nTMC CSP Master's Source Dish Vcc Config attribute value: \n{leaf_node_master.sourceDishVccConfig}"
)

dish_config_json file contents: 
{'interface': 'https://schema.skao.int/ska-mid-cbf-initsysparam/1.0', 'tm_data_sources': ['car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata'], 'tm_data_filepath': 'instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json'}
TMC CSP Master's Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "dish_parameters": {"SKA001": {"vcc": 1, "k": 1}, "SKA036": {"vcc": 2, "k": 1}, "SKA063": {"vcc": 3, "k": 1}, "SKA100": {"vcc": 4, "k": 1}}}

TMC CSP Master's Source Dish Vcc Config attribute value: 
{"interface": "https://schema.skao.int/ska-mid-cbf-initsysparam/1.0", "tm_data_sources": ["car://gitlab.com/ska-telescope/ska-telmodel-data?0.1.0-rc-mid-itf#tmdata"], "tm_data_filepath": "instrument/ska1_mid_itf/ska-mid-cbf-system-parameters.json"}


Before continuing, ensure this passes to make sure the DishVCC is set to allow for the On command to be sent.

In [23]:
if tmc_central_node.isDishVccConfigSet:
    print("DishVCC set, can continue!")
else:
    print("DishVCC not set, repeat last step.")

DishVCC set, can continue!


<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Load In BITE Config Data</h3>
    Now, if required, the BITE configuration data can be loaded in using the BITE notebook.
</div>
</body>

Set up the objects for controlling the telescope and subarray via OSO:

In [24]:
sub = SubArray(subarray_id)
tel = Telescope()

Now, while monitoring the Taranta dashboard, send the ON command to the telescope and monitor that the LURs Controller and subarrays come on (occasionally the ON command may not finish when everything has started):

In [25]:
tel.on()

1|2024-08-14T03:58:30.206Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track telescopeState of ska_mid/tm_central/central_node
1|2024-08-14T03:58:30.230Z|INFO|MainThread|execute|tango_executor.py#243||Executing command: <Command('ska_mid/tm_central/central_node', 'TelescopeOn', )>
1|2024-08-14T03:58:30.238Z|INFO|MainThread|wait_for_transition|common.py#338||Waiting for telescopeState to transition to ON
1|2024-08-14T03:58:30.240Z|INFO|MainThread|read_event|tango_executor.py#184||timeout None to wait for read an event from queue
1|2024-08-14T03:58:30.291Z|INFO|MainThread|read_event|tango_executor.py#184||timeout None to wait for read an event from queue
1|2024-08-14T03:58:58.895Z|INFO|MainThread|wait_for_transition|common.py#354||telescopeState reached target state ON


<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Generate BITE Data</h3>
    With the telescope on, section 3 of the BITE notebook can be run, to generate the BITE data that will be used during the LSTV replay.
</div>
</body>

### 2.1 Define Scheduling Block

Temporary work around for ingress issues:

In [26]:
os.environ["SKUID_URL"] = SKUID_POD
print("SKUID service set to: ", os.environ["SKUID_URL"])

SKUID service set to:  10.10.212.246:9870


Now, use the SKUID service to generate processing and execution block IDs from the scheduling block definition.

In [27]:
sb = load_sbd(SB_FILE)
# Configure sb items for current board:
sb.sdp_configuration.resources.receptors = receptor_ids
sb.dish_allocations.receptor_ids = receptor_ids
# for now run with 1 fsp
sb.csp_configurations[0].cbf.fsps[0].fsp_id = target_boards_list[0]
sb.csp_configurations[0].cbf.fsps[0].frequency_slice_id = target_boards_list[0]

sbi = create_sbi(sb)

sbi.sdp_configuration.execution_block.eb_id = exec_block_id

1|2024-08-14T03:59:24.960Z|INFO|MainThread|create_sbi|sb.py#28||New SBI ID mapping: sbi-mvp01-20200325-00001 -> sbi-t0001-20240814-00003
1|2024-08-14T03:59:24.965Z|INFO|MainThread|create_sbi|sb.py#41||New PB ID mapping: pb-test-20211111-00000 -> pb-t0001-20240814-00004
1|2024-08-14T03:59:24.965Z|INFO|MainThread|create_sbi|sb.py#41||New PB ID mapping: pb-test-20211111-00001 -> pb-t0001-20240814-00005


In [28]:
print(sb)
print(sbi)

interface='https://schema.skao.int/ska-oso-pdm-sbd/0.1' sbd_id='sbi-mvp01-20200325-00001' name=None description=None telescope=TelescopeType.SKA_MID metadata=Metadata(version=1, created_by='Liz Bartlett', created_on=datetime.datetime(2022, 3, 28, 15, 43, 53, 971548, tzinfo=TzInfo(UTC)), last_modified_by='Liz Bartlett', last_modified_on=datetime.datetime(2023, 11, 15, 10, 45, 3, 313536, tzinfo=TzInfo(UTC))) prj_ref=None activities={'observe': GitScript(function_args={'init': PythonArguments(args=[], kwargs={'subarray_id': 1}), 'main': PythonArguments(args=[], kwargs={})}, kind=ScriptKind.GIT, repo='https://gitlab.com/ska-telescope/oso/ska-oso-scripting', path='git://scripts/test_example_mid_script.py', branch='master', commit=None)} targets=[Target(target_id='Polaris Australis', pointing_pattern=PointingPattern(active=PointingKind.FIVE_POINT, parameters=[FivePointParameters(kind=PointingKind.FIVE_POINT, offset_arcsec=5.0), RasterParameters(kind=PointingKind.RASTER, row_length_arcsec=1.2

Using this, create the JSON for the scheduling block, subbing in the Kafka location, and validating it.

In [29]:
sb_json = SBDefinition.model_dump_json(sb)
sb_json = sb_json.replace("<KAFKA_HOST>", KAFKA_POD)
sb = SBDefinition.model_validate_json(sb_json)

## 3 Running the Observation

### 3.1 Assigning Resources

Now, the resources can be assigned to the telescope, either via getting the resources from the scheduling block:

In [30]:
assign_requests = pdm_transforms.create_cdm_assign_resources_request_from_scheduling_block(
    sub.id, sb
)

1|2024-08-14T03:59:33.030Z|INFO|MainThread|create_mid_assign_resources_request|wrapper.py#81||Setting dish : frozenset({'SKA001'}) 
1|2024-08-14T03:59:33.031Z|INFO|MainThread|convert_channels|sdp.py#293||Setting ChannelConfiguration id:vis_channels , spectral_windows:[SpectralWindow(spectral_window_id='fsp_1_channels', count=14880, start=0, freq_min=269000000.0, freq_max=469000000.0, stride=1, link_map=[(0, 0), (200, 1), (744, 2), (944, 3)])] 
1|2024-08-14T03:59:33.032Z|INFO|MainThread|convert_spectral_window|sdp.py#272||Setting channel attribute -> count:14880 , start:0 , stride:1, freq_min:269000000.0, freq_max:469000000.0 , link_map:[(0, 0), (200, 1), (744, 2), (944, 3)] 
1|2024-08-14T03:59:33.033Z|INFO|MainThread|convert_target_to_fieldconfiguration|sdp.py#347||Setting target attribute -> target_id:Polaris Australis , reference_coordinate:<EquatorialCoordinates: (21h08m47.92s -88d57m22.9s ICRS)> 
1|2024-08-14T03:59:33.034Z|INFO|MainThread|convert_target_to_fieldconfiguration|sdp.py

In [31]:
print(assign_requests)
print(assign_requests[0])
stripped_assign_requests = assign_requests[0]
stripped_assign_requests.interface = ""
stripped_assign_requests.dish.receptor_ids = receptor_ids
print(stripped_assign_requests)

[AssignResourcesRequest(subarray_id=1, dish=DishAllocation(receptor_ids=frozenset({'SKA001'})), sdp_config=SDPConfiguration(eb_id=None, max_length=None, scan_types=None, processing_blocks=[ProcessingBlockConfiguration(pb_id='pb-test-20211111-00000', workflow=None, parameters={'version': '0.5.0', 'num_scans': 5, 'additional_args': ['--thresh_width', '1.25'], 'kafka_topic': 'pointing_offset'}, dependencies=[PbDependency(pb_id='pb-test-20211111-00001', kind=['vis-receive'])], sbi_ids=[], script=ScriptConfiguration(kind='realtime', name='pointing-offset', version='0.5.0')), ProcessingBlockConfiguration(pb_id='pb-test-20211111-00001', workflow=None, parameters={'channels_per_port': 20, 'transport_protocol': 'udp', 'use_network_definition': True, 'extra_helm_values': {'receiver': {'options': {'reception': {'reset_time_indexing_after_each_scan': True}, 'telescope_model': {'telmodel_key': 'instrument/ska1_mid/layout/mid-layout.json'}}}}, 'pod_settings': [{'securityContext': {'runAsUser': 0, 'f

Now assign the resources to the subarray.

In [32]:
sub.assign_from_cdm(stripped_assign_requests, timeout=1000)

1|2024-08-14T03:59:38.556Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track obsState of ska_mid/tm_subarray_node/1
1|2024-08-14T03:59:38.566Z|INFO|MainThread|_get_id_from_params_or_generate_new_id|transactions.py#172||Generated transaction ID txn-t0001-20240814-000000007
1|2024-08-14T03:59:38.567Z|INFO|MainThread|log_entry|transactions.py#132||Transaction[txn-t0001-20240814-000000007]: Enter[AssignResources] with parameters [{}] marker[61972]
1|2024-08-14T03:59:38.568Z|INFO|MainThread|_call_and_wait_for_transition|common.py#178||Added Transaction_id to command: <Command('ska_mid/tm_central/central_node', 'AssignResources', '{"subarray_id": 1, "dish": {"receptor_ids": ["SKA001"]}, "sdp": {"processing_blocks": [{"pb_id": "pb-test-20211111-00000", "parameters": {"version": "0.5.0", "num_scans": 5, "additional_args": ["--thresh_width", "1.25"], "kafka_topic": "pointing_offset"}, "dependencies": [{"pb_id": "pb-test-20211111-00001", "kind": ["vis-receive"]}

Check that the visibility pod spins up correctly, with the signal displays.

In [36]:
!kubectl -n $namespace-sdp get pods | grep vis-receive

proc-pb-test-20211111-00001-vis-receive-00-0                   6/6     Running   1 (103s ago)   3m2s


<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Starting LSTV Replay</h3>
    If using BITE for data stream generation, the additional BITE notebook can be run at this point.
</div>
</body>

### 3.3 Run the Scan(s)

#### For single Scan

For a single scan, we load in the entire scan config and run via the subarray.

In [40]:
sub.scan()

1|2024-08-14T04:18:26.098Z|INFO|MainThread|_call_and_wait_for_transition|common.py#147||Using pub/sub to track obsState of ska_mid/tm_subarray_node/1
1|2024-08-14T04:18:26.099Z|INFO|MainThread|execute|tango_executor.py#243||Executing command: <Command('ska_mid/tm_subarray_node/1', 'Scan', '{"interface": "https://schema.skao.int/ska-tmc-scan/2.1", "scan_id": 3}')>
1|2024-08-14T04:18:26.110Z|INFO|MainThread|wait_for_transition|common.py#338||Waiting for obsState to transition to ObsState.SCANNING
1|2024-08-14T04:18:26.110Z|INFO|MainThread|read_event|tango_executor.py#184||timeout 65.0 to wait for read an event from queue
1|2024-08-14T04:18:26.116Z|INFO|MainThread|wait_for_transition|common.py#354||obsState reached target state ObsState.SCANNING
1|2024-08-14T04:18:26.116Z|INFO|MainThread|wait_for_transition|common.py#338||Waiting for obsState to transition to ObsState.READY
1|2024-08-14T04:18:26.116Z|INFO|MainThread|read_event|tango_executor.py#184||timeout 65.0 to wait for read an event 

#### For Multiple Scans with offset

For multiple scans, we have to offset and set up prior to each. To do this, we first create a list of configurations from our scheduling block before iterating over each and sending them.

In [38]:
scan_def_to_config_req_mapping = pdm_transforms.create_cdm_configure_request_from_scheduling_block(
    sb
)

{'scan_definitions': {'calibrator scan': ScanDefinition(scan_definition_id='calibrator scan', scan_duration=datetime.timedelta(seconds=60), target_ref='M83', target_beam_configuration_refs=[], dish_configuration_ref='dish config 123', scan_type_ref='pointing', csp_configuration_ref='csp config 123', pointing_correction=PointingCorrection.MAINTAIN), 'science scan': ScanDefinition(scan_definition_id='science scan', scan_duration=datetime.timedelta(seconds=60), target_ref='Polaris Australis', target_beam_configuration_refs=[], dish_configuration_ref='dish config 123', scan_type_ref='science', csp_configuration_ref='csp config 123', pointing_correction=PointingCorrection.MAINTAIN)}, 'targets': {'Polaris Australis': Target(target_id='Polaris Australis', pointing_pattern=PointingPattern(active=PointingKind.FIVE_POINT, parameters=[FivePointParameters(kind=PointingKind.FIVE_POINT, offset_arcsec=5.0), RasterParameters(kind=PointingKind.RASTER, row_length_arcsec=1.23, row_offset_arcsec=4.56, n_r

In [39]:
config_req = scan_def_to_config_req_mapping["science scan"]
print(config_req)
sub.configure_from_cdm(config_req[0], timeout=500)
sleep(5)
sub.scan()

[ConfigureRequest(pointing=PointingConfiguration(target=Target(ra=317.19966666666664, dec=-88.95636111111111, target_name='Polaris Australis', reference_frame='icrs', unit=('deg', 'deg'), ca_offset_arcsec=0.0, ie_offset_arcsec=0.0), correction=None), dish=DishConfiguration(receiver_band=<ReceiverBand.BAND_1: '1'>), sdp=SDPConfiguration(interface='https://schema.skao.int/ska-sdp-configure/0.4', scan_type='science'), csp=CSPConfiguration(interface='https://schema.skao.int/ska-csp-configure/2.0', subarray=SubarrayConfiguration(subarray_name='science period 23'), common=CommonConfiguration(config_id='csp config 123', frequency_band=<ReceiverBand.BAND_1: '1'>, subarray_id=1, band_5_tuning=None), cbf_config=CBFConfiguration(fsp_configs=[FSPConfiguration(fsp_id=1, function_mode=<FSPFunctionMode.CORR: 'CORR'>, frequency_slice_id=1, integration_factor=1, zoom_factor=0, channel_averaging_map=[(0, 2), (744, 0)], output_link_map=[(0, 0), (200, 1)], channel_offset=0, zoom_window_tuning=None)], vlbi

In [ ]:
print("...")
print(scan_def_to_config_req_mapping)
print("...")
for scan_def_id in sb.scan_sequence:
    config_req = scan_def_to_config_req_mapping[scan_def_id]
    print(config_req)
    print("--------------")
    for request in config_req:
        sub.configure_from_cdm(request, timeout=500)
        sleep(2)
        sub.scan()

## 4 Cleanup

<style>
    .alert {
        background-color: #1a1d21;
        border-style: dotted;
        border-color: #f0493e;
        color: #d1d2d3;
    }
</style>
<div class="alert">
    <h3>(BITE STEP) Stopping LSTV Replay</h3>
    Now with the initial (or only) scan done, if using BITE, run step 5 in the BITE notebook to stop the LSTV Replay.
</div>
</body>

Once the scans are done, clear out the configuration from the subarray by sending the END command, followed by releasing the resources it used:

In [ ]:
sub.end()
sleep(10)
sub.release()

The telescope can now be turned off.

In [ ]:
tel.off()